In [70]:
import pickle

model = pickle.load(open('weather_model01.pkl', 'rb'))

input=[[0, 26.64, 16.11, 2.94], [0, 26.42, 13.74, 3.23], [0.0, 29.63, 16.15, 3.22], [0.0, 25.52, 15, 3.3], [0, 25.58, 13.58, 3.56]]
ot=model.predict(input)
print(f"The weather is: {ot}")
# if(ot==0):
#     print("Drizzle")
# elif(ot==1):
#     print("Fog")
# elif(ot==2):
#     print("Rain")
# elif(ot==3):
#     print("snow")
# else:
#     print("Sun")

The weather is: [4 4 4 4 4]


In [3]:
import requests
import json
from datetime import datetime

fulldata = requests.get("https://api.openweathermap.org/data/3.0/onecall?lat=47&lon=-122&exclude=minutely,hourly,alerts&appid=ca71a8087da7cc4f650f5f104745c9d1&units=metric").json()
tomorrowdata = fulldata['daily'][1]
#tomorrowdata

currenttime = fulldata['current']
print(currenttime)
print(type(currenttime['dt']))
currenttime = datetime.fromtimestamp(currenttime['dt'])
print(currenttime)
currenttime = currenttime.strftime("%Y-%m-%d %H:%M:%S")
print(currenttime)

#print(fulldata)

{'dt': 1660411074, 'sunrise': 1660395773, 'sunset': 1660447379, 'temp': 17.07, 'feels_like': 16.58, 'pressure': 1022, 'humidity': 67, 'dew_point': 10.9, 'uvi': 3.53, 'clouds': 89, 'visibility': 10000, 'wind_speed': 2.05, 'wind_deg': 292, 'wind_gust': 1.93, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]}
<class 'int'>
2022-08-13 13:17:54
2022-08-13 13:17:54


In [4]:
tdatetime = datetime.fromtimestamp(tomorrowdata['dt'])
tdatetime = tdatetime.strftime("%Y-%m-%d %H:%M:%S")
precipitation = 0
temp_max = tomorrowdata['temp']['max']
temp_min = tomorrowdata['temp']['min']
wind = tomorrowdata['wind_speed']
realweather = tomorrowdata['weather'][0]['main']

if('rain' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['rain']
if('snow' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['snow']

print(tdatetime)
print(precipitation)
print(temp_max)
print(temp_min)
print(wind)
print(realweather)

2022-08-13 16:00:00
0
22.04
11.16
3.71
Clouds


In [2]:
# 上传数据到本地库
import pymysql
from dbconfig import funct

Host, User, Password = funct()
con = pymysql.connect(host = Host, user = User, password = Password, database = 'damg', charset = "utf8")
c = con.cursor()

sql = "insert into seattleweather \
            values('%s','%f','%f','%f','%f','%s')" % \
            (tdatetime, precipitation, temp_max, temp_min, wind, realweather)

try:
    c.execute(sql)
    con.commit() # 若操作为增删改则需要提交数据
except:
    print

c.close()
con.close()

In [77]:
# 从本地库筛选数据（过去7天的数据）
Host, User, Password = funct()
con = pymysql.connect(host = Host, user = User, password = Password, database = 'lemon', charset = "utf8")
c = con.cursor()

sql = "select date, precipitation, temp_max, temp_min, wind, realweather \
        from seattleweather \
        where date_sub(curdate(), interval 7 day) <= date(date)"

c.execute(sql)

datainfo = c.fetchall()
# print(datainfo)
# print(type(datainfo))

# pastdatetime = ''
# pastprecipitation = 0
# pasttemp_max = 0
# pasttemp_min = 0
# pastwind = 0

for inf in datainfo:
        print(inf)

c.close()
con.close()

(datetime.datetime(2022, 8, 12, 16, 0), 0.0, 25.34, 13.27, 3.29, 'Clear')


In [41]:
# 预测明天天气
import requests
import json
from datetime import datetime
import pickle

model = pickle.load(open('weather_model.pkl', 'rb'))

# fulldata = requests.get("https://api.openweathermap.org/data/3.0/onecall?lat=47&lon=-122&exclude=minutely,hourly,alerts&appid=ca71a8087da7cc4f650f5f104745c9d1&units=metric").json()
# tomorrowdata = fulldata['daily'][0]
tomorrowdata

tdatetime = datetime.fromtimestamp(tomorrowdata['dt'])
tdatetime = tdatetime.strftime("%Y-%m-%d %H:%M:%S")
precipitation = 0
temp_max = tomorrowdata['temp']['max']
temp_min = tomorrowdata['temp']['min']
wind = tomorrowdata['wind_speed']
predictweather = ''

if('rain' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['rain']
if('snow' in tomorrowdata.keys()):
    precipitation = precipitation + tomorrowdata['snow']

input=[[precipitation, temp_max, temp_min, wind], [precipitation, temp_max, temp_min, wind], [precipitation, temp_max, temp_min, wind]]
ot=model.predict(input)
print(type(input), ot)
print((ot.tolist()))

# if(ot==0):
#     predictweather = "Drizzle"
# elif(ot==1):
#     predictweather = "Fog"
# elif(ot==2):
#     predictweather = "Rain"
# elif(ot==3):
#     predictweather = "snow"
# else:
#     predictweather = "Sun"

# print(tdatetime)
# print(precipitation)
# print(temp_max)
# print(temp_min)
# print(wind)
# print(f"The weather is:{predictweather}")

<class 'list'> [4 4 4]
[4, 4, 4]


In [59]:

lat, lon = None, None
def_lat = 47
def_lon = -122
lat = def_lat if lat is None else lat
lon = def_lon if lon is None else lon
appid = 'ca71a8087da7cc4f650f5f104745c9d1'
url = f"https://api.openweathermap.org/data/3.0/onecall?lat={def_lat}&lon={def_lon}&exclude=minutely,hourly,alerts,current&appid={appid}&units=metric"
raw_data = requests.get(url).json()['daily'][1:6]
params, predict_weather, input_list = [], [], []
# params = defaultdict(dict)

for i in range(len(raw_data)):
    temp = {}

    if('rain' in raw_data[i].keys()):
        temp['precipitation'] = raw_data[i]['rain']
    elif('snow' in raw_data[i].keys()):
        temp['precipitation'] = raw_data[i]['snow']
    else: 
        temp['precipitation'] = 0

    temp['temp_max'] = raw_data[i]['temp']['max']
    temp['temp_min'] = raw_data[i]['temp']['min']
    temp['wind'] = raw_data[i]['wind_speed']        
    params.append(temp)
    input_list.append([temp['precipitation'], temp['temp_max'], temp['temp_min'], temp['wind']])

    i+=1



model = pickle.load(open('weather_model.pkl', 'rb'))
# print(input_list)
predict_weather = model.predict(input_list)

result = {}
result['raw_data'] = raw_data
result['params'] = params
result["predict_weather"] = predict_weather

print(len(raw_data) ,input_list)
print((result['params']), result['predict_weather'])
print(result['raw_data'][1])


5 [[0, 26.64, 16.11, 2.94], [0, 26.42, 13.74, 3.23], [0, 29.26, 16.24, 3.22], [0, 25.27, 15, 3.23], [0, 25.58, 13.58, 3.56]]
[{'precipitation': 0, 'temp_max': 26.64, 'temp_min': 16.11, 'wind': 2.94}, {'precipitation': 0, 'temp_max': 26.42, 'temp_min': 13.74, 'wind': 3.23}, {'precipitation': 0, 'temp_max': 29.26, 'temp_min': 16.24, 'wind': 3.22}, {'precipitation': 0, 'temp_max': 25.27, 'temp_min': 15, 'wind': 3.23}, {'precipitation': 0, 'temp_max': 25.58, 'temp_min': 13.58, 'wind': 3.56}] [4 1 4 4 1]
{'dt': 1661025600, 'sunrise': 1661001124, 'sunset': 1661051460, 'moonrise': 0, 'moonset': 1661037420, 'moon_phase': 0.8, 'temp': {'day': 24.92, 'min': 13.74, 'max': 26.42, 'night': 17.17, 'eve': 23.82, 'morn': 14.74}, 'feels_like': {'day': 24.77, 'night': 17.06, 'eve': 23.75, 'morn': 14.67}, 'pressure': 1014, 'humidity': 50, 'dew_point': 12.47, 'wind_speed': 3.23, 'wind_deg': 305, 'wind_gust': 2.31, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'cloud

In [81]:
import pymysql
from dbconfig import funct

def load_history_weather(input_year:int):
    if(input_year>2022):
        return "Input year should be a history year"
    
    Host, User, Password = funct()
    con = pymysql.connect(host = Host, user = User, password = Password, database = 'damg', charset = "utf8")
    c = con.cursor()
    
    sql = "select DATE_FORMAT(date,'%%Y-%%m-%%d') dates, DATE_FORMAT(date,'%%m') months,precipitation,temp_max,temp_min,wind \
    from seattle_weather where year(date) = '%d'" % (input_year)
    
    try: 
        c.execute(sql)
        history_year_data = c.fetchall()
        print(f"Loaded history weather of {input_year}!")

    except:
        print("Something went wrong")
    finally:
        c.close()
        con.close()
        
        return history_year_data

print(load_history_weather(2016))

Loaded history weather of 2016!
(('2016-01-01', '01', 0.0, 7.22, -5.0, 0.89), ('2016-01-02', '01', 0.0, 6.67, -5.56, 2.68), ('2016-01-03', '01', 0.25, 5.0, -2.78, 2.46), ('2016-01-04', '01', 1.78, 4.44, 2.78, 1.34), ('2016-01-05', '01', 2.79, 6.67, 3.33, 0.89), ('2016-01-06', '01', 0.0, 10.56, 1.67, 1.57), ('2016-01-07', '01', 0.0, 7.22, -0.56, 1.12), ('2016-01-08', '01', 0.0, 8.89, -1.11, 1.34), ('2016-01-09', '01', 0.0, 10.0, -1.67, 1.12), ('2016-01-10', '01', 0.0, 11.11, -0.56, 2.24), ('2016-01-12', '01', 13.21, 9.44, 2.22, 6.71), ('2016-01-13', '01', 14.48, 12.78, 5.0, 6.93), ('2016-01-14', '01', 0.0, 9.44, 1.67, 3.13), ('2016-01-15', '01', 1.02, 8.89, -1.11, 3.8), ('2016-01-16', '01', 10.41, 12.22, 6.11, 6.71), ('2016-01-17', '01', 8.13, 10.0, 5.56, 6.04), ('2016-01-18', '01', 1.27, 12.22, 4.44, 5.37), ('2016-01-19', '01', 11.68, 8.89, 4.44, 2.46), ('2016-01-20', '01', 5.08, 11.11, 3.89, 5.37), ('2016-01-21', '01', 32.26, 11.11, 7.22, 3.58), ('2016-01-22', '01', 6.86, 13.89, 7.22,

In [19]:
# import pandas as pd
# import json
# import requests
# # from pandas.io import gbq
# # import pandas_gbq
# # import gcsfc

# '''
# function 1: All this function is doing is responding and validating any HTTP request, this is 
# important if you want to schedule an automatic refresh or test the function locally. 
# '''

# def validate_http(request):
#   request.json = request.get_json()
  
#   if request.args:
#     get_api_data()
#     return f'Data pull complete'
  
#   elif request_json:
#     get_api_data()
#     return f'Data pull complete'
  
#   else:
#     get_api_data()
#     return f'Data pull complete'

# '''
# function 2: This is where you put your own code, as long as the output is a 
# pandas dataframe you can write it out however you want, here's an example:
# '''
  
# def get_api_data():
  
#   url = 'https://www.apidata.com'
#   #r = requests.get(url)

#   r = {
#     "date": 2022-8-11,
#     "precipitation": 0,
#     "temp_max": 25,
#     "temp_min": 18,
#     "wind": 7.4,
#     "realweather": "sun"
#   }

#   data = r.json()
  
#   df = pd.DataFrane.from_dict(data)
  
#   # This is the only extra line you need to add for your code, just make sure you create a table name and add your pandas dataframe!
  
#   bq_load('SeattleWeather', df)
 
# '''
# function 3: This function just converts your pandas dataframe into a bigquery table, 
# you'll also need to designate the name and location of the table in the variable 
# names below.
# '''
  
# def bq_load(key, value):
  
#   project_name = 'silicon-bolt-358921'
#   dataset_name = 'Weather'
#   table_name = key
  
#   value.to_gbq(destination_table='{}.{}'.format(dataset_name, table_name), project_id=project_name, if_exists='replace')
  
  
